In [1]:
import pandas as pd
import numpy as np
from gensim import corpora
from gensim.models import VocabTransform

from common import *
import os
import glob
from joblib import Parallel, delayed
import itertools
import multiprocessing
import copy 

cpu_count = multiprocessing.cpu_count()
DATA_FOLDER = '../data/'

Using TensorFlow backend.


In [ ]:
# all_docs = glob.iglob(os.path.join(DATA_FOLDER, 'FIPS') + '/**/*.txt', recursive=True)
# corpus_file = open(DATA_FOLDER + 'corpus.txt', 'w')
# for item in all_docs:
#     corpus_file.write("%s\n" % os.path.relpath(item, DATA_FOLDER))

In [ ]:
# from nltk.tokenize.toktok import ToktokTokenizer 
# toktok = ToktokTokenizer()
# toktok.tokenize('устройство по п . 3 , отличать тем , что оно снабжать устанавливать в головка винт , в который размещать шарик и ролик .  . ')

In [ ]:
# nltk.download()

In [ ]:
all_docs = get_all_docs(DATA_FOLDER)

In [ ]:
parallelizer = Parallel(n_jobs=cpu_count)

# this iterator returns the functions to execute for each task
tasks_iterator = ( delayed(save_corpus)(list_block, os.path.join(DATA_FOLDER, 'BoW'), i) for 
                  i, list_block in enumerate(grouper(len(all_docs)//100, all_docs)) ) 
result = parallelizer( tasks_iterator )

In [2]:
corps = glob.glob(join(DATA_FOLDER, 'BoW/*corpus.mm'))
dicts = glob.glob(join(DATA_FOLDER, 'BoW/*.dict'))
corps.sort(key=natural_keys)
dicts.sort(key=natural_keys)
pairs = list(zip(dicts, corps))
result = pairs

In [3]:
%%time
merged_corpus = None
for dic_name, corp_name in result:
    cur_dict = corpora.Dictionary.load(dic_name)
    cur_corp = corpora.MmCorpus(corp_name)
    
    print(dic_name)
    if not merged_corpus:
        dict1 = cur_dict
        merged_corpus = cur_corp
        continue

    cur_to_dict1 = dict1.merge_with(cur_dict)
    merged_corpus = itertools.chain(merged_corpus, cur_to_dict1[cur_corp])
    
dict1.save(join(DATA_FOLDER, 'old.dict'))
corpora.MmCorpus.serialize(join(DATA_FOLDER, 'corpus.mm'), merged_corpus)

../data/BoW/0.dict
../data/BoW/1.dict
../data/BoW/2.dict
../data/BoW/3.dict
../data/BoW/4.dict
../data/BoW/5.dict
../data/BoW/6.dict
../data/BoW/7.dict
../data/BoW/8.dict
../data/BoW/9.dict
../data/BoW/10.dict
../data/BoW/11.dict
../data/BoW/12.dict
../data/BoW/13.dict
../data/BoW/14.dict
../data/BoW/15.dict
../data/BoW/16.dict
../data/BoW/17.dict
../data/BoW/18.dict
../data/BoW/19.dict
../data/BoW/20.dict
../data/BoW/21.dict
../data/BoW/22.dict
../data/BoW/23.dict
../data/BoW/24.dict
../data/BoW/25.dict
../data/BoW/26.dict
../data/BoW/27.dict
../data/BoW/28.dict
../data/BoW/29.dict
../data/BoW/30.dict
../data/BoW/31.dict
../data/BoW/32.dict
../data/BoW/33.dict
../data/BoW/34.dict
../data/BoW/35.dict
../data/BoW/36.dict
../data/BoW/37.dict
../data/BoW/38.dict
../data/BoW/39.dict
../data/BoW/40.dict
../data/BoW/41.dict
../data/BoW/42.dict
../data/BoW/43.dict
../data/BoW/44.dict
../data/BoW/45.dict
../data/BoW/46.dict
../data/BoW/47.dict
../data/BoW/48.dict
../data/BoW/49.dict
../data/Bo

### Filter out tokens that appear in less than 5 documents (absolute number) or more than 50% documents

In [ ]:
%%time
# filter the dictionary
old_dict = corpora.Dictionary.load(join(DATA_FOLDER, 'old.dict'))
new_dict = copy.deepcopy(old_dict)
new_dict.filter_extremes(no_below=3, keep_n=None)
new_dict.save(join(DATA_FOLDER, 'filtered.dict'))

In [ ]:
%%time
# now transform the corpus
corpus = corpora.MmCorpus(join(DATA_FOLDER, 'corpus.mm'))
old2new = {old_dict.token2id[token]:new_id for new_id, token in new_dict.iteritems()}
vt = VocabTransform(old2new)

In [ ]:
%%time
corpora.MmCorpus.serialize(join(DATA_FOLDER, 'filtered_corpus.mm'), vt[corpus], id2word=new_dict)

In [55]:
!cp ../data/*corpus* ~/Yandex.Disk
!cp ../data/*.dict ~/Yandex.Disk
!cp ../data/all_docs.txt ~/Yandex.Disk

cp: cannot stat '../data/*corpus*': No such file or directory
cp: cannot stat '../data/*.dict': No such file or directory
cp: cannot stat '../data/all_docs.txt': No such file or directory
